In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

StatementMeta(smallSpark, 38, 1, Finished, Available)

### Get the dissertation webpage

In [278]:
url_extension = 'non_oap_facpubs/index.11.html'
LINK_INDICATOR = "facpubs/"
URL = f"https://digitalcommons.uri.edu/{url_extension}"
page = requests.get(URL)

StatementMeta(smallSpark, 38, 278, Finished, Available)

### Parse out specific data from the page content

In [279]:
soup = BeautifulSoup(page.content, "html.parser")

StatementMeta(smallSpark, 38, 279, Finished, Available)

Extract the link from each article listing

In [280]:
main_div = soup.find(id="series-home")

StatementMeta(smallSpark, 38, 280, Finished, Available)

In [281]:
# main_div

StatementMeta(smallSpark, 38, 281, Finished, Available)

In [282]:
# Get the link to the page containing more information on this dissertation.
links = main_div.find_all("a") # , class_="article-listing")
d_info_links = [d for d in links if LINK_INDICATOR in str(d)]

StatementMeta(smallSpark, 38, 282, Finished, Available)

In [283]:
d_info_links

StatementMeta(smallSpark, 38, 283, Finished, Available)

[<a href="https://digitalcommons.uri.edu/bps_facpubs/344">Saudiarabicains A-E, bioactive 19-acetoxyingol diterpenoids from Euphorbia saudiarabica</a>,
 <a href="https://digitalcommons.uri.edu/nursing_facpubs/87">Building a culture of safety and quality: The paradox of measurement</a>,
 <a href="https://digitalcommons.uri.edu/bio_facpubs/607">Editorial: Integrating Predation Risk Across Scales: from Neurons to Ecosystems and Milliseconds to Generations</a>,
 <a href="https://digitalcommons.uri.edu/psy_facpubs/67">Avian Bornaviral Ganglioneuritis: Current Debates and Unanswered Questions</a>,
 <a href="https://digitalcommons.uri.edu/cmb_facpubs/181">Thermal range and physiological tolerance mechanisms in two shark species from the northwest Atlantic</a>,
 <a href="https://digitalcommons.uri.edu/gsofacpubs/705">Small phytoplankton dominate western North Atlantic biomass</a>,
 <a href="https://digitalcommons.uri.edu/math_facpubs/138">Bounds on the localization number</a>,
 <a href="https:/

In [284]:
papers = [{'link':str(l).split('\"')[1]} for l in d_info_links]
papers = papers[:-9] # Remove a few erroneous page links

StatementMeta(smallSpark, 38, 284, Finished, Available)

In [285]:
papers[-10:]

StatementMeta(smallSpark, 38, 285, Finished, Available)

[{'link': 'https://digitalcommons.uri.edu/gsofacpubs/728'},
 {'link': 'https://digitalcommons.uri.edu/gsofacpubs/753'},
 {'link': 'https://digitalcommons.uri.edu/bio_facpubs/246'},
 {'link': 'https://digitalcommons.uri.edu/phys_facpubs/326'},
 {'link': 'https://digitalcommons.uri.edu/phys_facpubs/325'},
 {'link': 'https://digitalcommons.uri.edu/phys_facpubs/335'},
 {'link': 'https://digitalcommons.uri.edu/cba_facpubs/212'},
 {'link': 'https://digitalcommons.uri.edu/psy_facpubs/76'},
 {'link': 'first'},
 {'link': 'prev'}]

Get each linked page and extract each desired data point from that page

In [286]:
db_entries = []

for d in papers[:-2]:
    try:
        url = d['link']
        _d_page = requests.get(url)
        _d_soup = BeautifulSoup(_d_page.content, "html.parser")
        _main_div = _d_soup.find("div", id="alpha")
        # print(_main_div)
        
        researcher = str(_main_div.find_all("strong")[0])[8:-9] # The researcher
        who = researcher # NOTE: Add advisor?

        _date_div = _d_soup.find("div", id="publication_date")
        date_awarded = str(_date_div.find_all("p")[0])[3:-4]
        when = date_awarded

        _title_div = _d_soup.find("div", id="title")
        title = str(_title_div.find_all("p")[0]).split('>')[2][:-3]
        _abstract_div = _d_soup.find("div", id="abstract")
        abstract_paras = _abstract_div.find_all("p")
        abstract_paras = [str(para)[3:-4] for para in abstract_paras] # Clean tags from data
        # abstract_str = ''
        for p in abstract_paras: # TODO: Each paragraph should become its own entry
            # abstract_str += p + '\n'
            db_entries.append(
                {
                    'link': url,
                    'who': who,
                    'what': 'Research article title: ' + title + ', Paragraph from abstract: ' + p,
                    'when': date_awarded
                }
            )



    except Exception as e:
        print(e)

    # print("\n\nWhat: ", d['what'])
    # print("\n\nWho: ", d['who'])
    # print("\n\nWhen: ", date_awarded)


StatementMeta(smallSpark, 38, 286, Finished, Available)

'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'


In [287]:
len(db_entries)

StatementMeta(smallSpark, 38, 287, Finished, Available)

126

In [288]:
db_entries[-10:]

StatementMeta(smallSpark, 38, 288, Finished, Available)

[{'link': 'https://digitalcommons.uri.edu/gsofacpubs/753',
  'who': 'Ian T. Cousins',
  'what': 'Research article title: The High Persistence of PFAS is Sufficient for Their Management as a Chemical Class, Paragraph from abstract: Per- and polyfluoroalkyl substances (PFAS) are a class of synthetic organic substances with diverse structures, properties, uses, bioaccumulation potentials and toxicities. Despite this high diversity, all PFAS are alike in that they contain perfluoroalkyl moieties that are extremely resistant to environmental and metabolic degradation. The vast majority of PFAS are therefore either non-degradable or transform ultimately into stable terminal transformation products (which are still PFAS). Under the European chemicals regulation this classifies PFAS as very persistent substances (vP). We argue that this high persistence is sufficient concern for their management as a chemical class, and for all “non-essential” uses of PFAS to be phased out. The continual relea

In [289]:
d_df = pd.DataFrame.from_dict(db_entries).dropna()

StatementMeta(smallSpark, 38, 289, Finished, Available)

In [290]:
d_df

StatementMeta(smallSpark, 38, 290, Finished, Available)

,link,who,what,when
0,https://digitalcommons.uri.edu/bps_facpubs/344,Abdullatif Bin Muhsinah,"Research article title: , Paragraph from abstr...",8-6-2020
1,https://digitalcommons.uri.edu/nursing_facpubs/87,Katherine Bliss,"Research article title: , Paragraph from abstr...",7-1-2020
2,https://digitalcommons.uri.edu/psy_facpubs/67,Susan Boatright-Horowitz,Research article title: Avian Bornaviral Gangl...,1-19-2020
3,https://digitalcommons.uri.edu/cmb_facpubs/181,Abigail B. Bockus,"Research article title: , Paragraph from abstr...",4-1-2020
4,https://digitalcommons.uri.edu/gsofacpubs/705,Luis M. Bolanos,Research article title: Small phytoplankton do...,3-30-2020
...,...,...,...,...
121,https://digitalcommons.uri.edu/cba_facpubs/212,W. E.Douglas Creed,"Research article title: , Paragraph from abstr...",3-1-2020
122,https://digitalcommons.uri.edu/psy_facpubs/76,Robert M. Cronin,Research article title: Association between Ho...,6-2020
123,https://digitalcommons.uri.edu/psy_facpubs/76,Robert M. Cronin,Research article title: Association between Ho...,6-2020
124,https://digitalcommons.uri.edu/psy_facpubs/76,Robert M. Cronin,Research article title: Association between Ho...,6-2020


In [291]:
file_name = os.getenv("FILE_PATH")
file_name

StatementMeta(smallSpark, 38, 291, Finished, Available)

'abfss://iifs@iidatalake.dfs.core.windows.net/Exploratory/URI_Resources/OpenAcademicResources_non_oap_facpubs_index_11_html.csv'

In [292]:
d_df.to_csv(file_name)

StatementMeta(smallSpark, 38, 292, Finished, Available)